# NATURAL LANGUAGE PROCESSING

## Setup

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.4.0

# install spark-nlp
%pip install spark-nlp==5.1.3

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# # Import pyspark and build Spark session
# from pyspark.sql import SparkSession

# # Import pyspark and build Spark session
# spark = SparkSession.builder \
#     .appName("Spark NLP")\
#     .master("local[*]")\
#     .config("spark.driver.memory","16G")\
#     .config("spark.driver.maxResultSize", "0") \
#     .config("spark.kryoserializer.buffer.max", "2000M")\
#     .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3")\
#     .getOrCreate()

# print(spark.version)

# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

# spark = (
#     SparkSession.builder.appName("PySparkApp")
#     .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
#     .config(
#         "fs.s3a.aws.credentials.provider",
#         "com.amazonaws.auth.ContainerCredentialsProvider",
#     )
#     .getOrCreate()
# )

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-71ed2c52-d21b-40a0-8997-69ea82d57eff;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

3.4.0


## Import Libraries

In [3]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
print(f"Spark version: {spark.version}")
print(f"sparknlp version: {sparknlp.version()}")

Spark version: 3.4.0
sparknlp version: 5.1.3


## Import Data

In [6]:
%%time
bucket = "project-group34"
session = sagemaker.Session()
output_prefix_data_comments = "project/comments/yyyy=2021"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
print(f"reading comments from {s3_path}")
comments = spark.read.parquet(s3_path, header=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://project-group34/project/comments/yyyy=2021
CPU times: user 25.2 ms, sys: 671 µs, total: 25.9 ms
Wall time: 829 ms


In [7]:
# comments = comments.cache()

In [6]:
comments.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)



In [8]:
# display a subset of columns
comments.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality").show()

+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|       subreddit|         author|                body| parent_id|     id|        created_utc|score|controversiality|
+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|    Animesuggest|        Athenza|{Now and Then, He...| t3_m3ygv3|gqscelh|2021-03-13 10:15:52|    2|               0|
|    Animesuggest|       Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|gqscf1z|2021-03-13 10:16:05|    1|               0|
|    Animesuggest|      [deleted]|           [deleted]| t3_m3vnjl|gqscjse|2021-03-13 10:18:25|    1|               0|
|MovieSuggestions|      katnip_fl|       Jacobs Ladder| t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|
|    Animesuggest|        Athenza|{Kino no Tabi: Th...| t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|    1|               0|
|    Animesuggest|    Dropsoftime|Try Mahouka kouko...| 

### DATA PROCESSING

In [9]:
# Filter out rows where 'body' or 'author' is '[deleted]'
comments_filtered = comments.filter((comments.body != '[deleted]') & (comments.author != '[deleted]'))

# Show the filtered DataFrame
comments_filtered = comments_filtered.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality")

In [36]:
comments_filtered.show()

+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|       subreddit|         author|                body| parent_id|     id|        created_utc|score|controversiality|
+----------------+---------------+--------------------+----------+-------+-------------------+-----+----------------+
|    Animesuggest|        Athenza|{Now and Then, He...| t3_m3ygv3|gqscelh|2021-03-13 10:15:52|    2|               0|
|    Animesuggest|       Roboragi|**Ima, Soko ni Ir...|t1_gqscelh|gqscf1z|2021-03-13 10:16:05|    1|               0|
|MovieSuggestions|      katnip_fl|       Jacobs Ladder| t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|
|    Animesuggest|        Athenza|{Kino no Tabi: Th...| t3_m3xpu6|gqscnqz|2021-03-13 10:20:26|    1|               0|
|    Animesuggest|    Dropsoftime|Try Mahouka kouko...| t3_m43dco|gqscnr8|2021-03-13 10:20:26|    3|               0|
|MovieSuggestions|   alienstabler|[Holes (2003)](ht...| 

In [10]:
comments_filtered_movies = comments_filtered.where(col("subreddit").isin("MovieSuggestions"))

In [12]:
# # Define the pipeline stages
# document_assembler = DocumentAssembler() \
#     .setInputCol("body") \
#     .setOutputCol("document")

# sentence_detector = SentenceDetector() \
#     .setInputCols(["document"]) \
#     .setOutputCol("sentence")

# tokenizer = Tokenizer() \
#     .setInputCols(["sentence"]) \
#     .setOutputCol("token")

# # Use a pretrained embeddings model, for example, BERT
# embeddings = BertEmbeddings.pretrained("bert_base_cased", "en") \
#     .setInputCols(["sentence", "token"]) \
#     .setOutputCol("embeddings")

# ner_model = NerDLModel.pretrained("ner_dl_bert", "en") \
#     .setInputCols(["sentence", "token", "embeddings"]) \
#     .setOutputCol("ner")

# ner_converter = NerConverter() \
#     .setInputCols(["sentence", "token", "ner"]) \
#     .setOutputCol("ner_chunk")

# # Build the pipeline
# nlp_pipeline = Pipeline(stages=[
#     document_assembler,
#     sentence_detector,
#     tokenizer,
#     embeddings,
#     ner_model,
#     ner_converter
# ])

# # Apply the pipeline to your DataFrame
# model = nlp_pipeline.fit(comments_filtered_movies)
# result = model.transform(comments_filtered_movies)

bert_base_cased download started this may take some time.
Approximate size to download 384.9 MB
[OK!]
ner_dl_bert download started this may take some time.
Approximate size to download 15.4 MB
[OK!]


### MOVIE SUGGESTIONS EXTRACTION USING NER

In [ ]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import SentenceDetector, Tokenizer, WordEmbeddingsModel, NerDLModel, NerConverter
from pyspark.ml import Pipeline

# Define the pipeline stages
document_assembler = DocumentAssembler() \
    .setInputCol("body") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

# Use GloVe embeddings
embeddings = WordEmbeddingsModel.pretrained("glove_100d", "en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

# Use a lighter NER model
ner_model = NerDLModel.pretrained("ner_dl", "en") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

# Build the pipeline
nlp_pipeline = Pipeline(stages=[
    document_assembler,s
    sentence_detector,
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])

# Apply the pipeline to your DataFrame
model = nlp_pipeline.fit(comments_filtered_movies)
result = model.transform(comments_filtered_movies)


In [13]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, ArrayType
import pyspark.sql.functions as F

# Define a UDF to filter and extract movie names
def extract_movies(chunks):
    movie_names = [chunk.result for chunk in chunks if chunk.metadata['entity'] in ['PERSON', 'ORG']]
    return movie_names

extract_movie_names_udf = udf(extract_movies, ArrayType(StringType()))

# Apply the UDF to the DataFrame
movies_df = result.withColumn("movie_names", extract_movie_names_udf(F.col("ner_chunk")))

# # Display the results
# movies_df.select("body", "movie_names").show()

In [15]:
movies_df.show(5)

+----------------+--------------+--------------------+---------+-------+-------------------+-----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|       subreddit|        author|                body|parent_id|     id|        created_utc|score|controversiality|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|    movie_names|
+----------------+--------------+--------------------+---------+-------+-------------------+-----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|MovieSuggestions|     katnip_fl|       Jacobs Ladder|t3_m3rw47|gqscl5i|2021-03-13 10:19:07|    2|               0|[{document, 0, 12...|[{document, 0, 12...|[{token, 0, 5, Ja...|[{word_embeddings...|[{named_entity, 0...|[{chun

In [ ]:
movies_df.write.parquet("s3a://project-group34/project/suggestions/movies/yyyy=2021/", mode="overwrite")

### MOVIE SUGGESTIONS EXTRACTION USING REGEX

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, StructType, StructField

# import spacy
import re

# # Load spaCy model
# nlp = spacy.load("en_core_web_sm")

# # Define schema for the UDF output
# movie_schema = StructType([
#     StructField("movie_positions", ArrayType(ArrayType(StringType()))),
#     StructField("movie_names", ArrayType(StringType()))
# ])

# # UDF to extract movie names
# @udf(movie_schema)
# def extract_movie_names_udf(text):
#     doc = nlp(text)
#     movie_positions = []
#     movie_names = []

#     for ent in doc.ents:
#         if ent.label_ == "ORG" or ent.label_ == "PERSON":
#             movie_positions.append([ent.start_char, ent.end_char])
#             movie_names.append(ent.text)

#     return (movie_positions, movie_names)

# UDF to remove movie names
@udf(StringType())
def remove_movie_names_udf(text, movie_names):
    if movie_names:
        for name in movie_names:
            text = text.replace(name, ' ')
        return ' '.join(text.split())
    else:
        return text

# UDF to extract movie names using regex
@udf(ArrayType(StringType()))
def extract_movie_names_regex_udf(text, movie_names):
    movie_name_pattern = r'(?:\"([^\"]+)\"|([A-Z][a-z]*(?:\s+(?:[a-z]+\s+)*[A-Z][a-z]*)*)(?: \(\d{4}\))?)'

    movie_matches = re.findall(movie_name_pattern, text)
    movies = [match[0] or match[1] or match[2] for match in movie_matches]
    return movie_names + movies

# Remove movie names from the 'body' text
df_removed_movie_names = movies_df.withColumn("body_no_movies", remove_movie_names_udf(movies_df["body"], movies_df["movie_names"]))

# If you still want to use the regex method to supplement the NER extraction
df_final = df_removed_movie_names.withColumn("additional_movie_names", extract_movie_names_regex_udf(df_removed_movie_names["body_no_movies"], df_removed_movie_names["movie_names"]))


In [17]:
df_final.select("body", "movie_names", "additional_movie_names").show()

+--------------------+--------------------+----------------------+
|                body|         movie_names|additional_movie_names|
+--------------------+--------------------+----------------------+
|       Jacobs Ladder|     [Jacobs Ladder]|       [Jacobs Ladder]|
|[Holes (2003)](ht...|                  []|               [Holes]|
|V for Vendetta\n\...|                  []|  [V for Vendetta\n...|
|Synchronic was co...|                  []|          [Synchronic]|
|Climax!!!!! Sound...|       [Climax!!!!!]|  [Climax!!!!!, Sou...|
|• Sorry To Bother...|                  []|  [Sorry To Bother ...|
|I would also sugg...|                  []|  [I would also sug...|
|Midsommar. Was no...|         [Midsommar]|  [Midsommar, Was n...|
|Reservoir Dogs\n\...|                  []|  [Reservoir Dogs\n...|
|             Hoodlum|           [Hoodlum]|             [Hoodlum]|
|Crime/Thriller:\n...|[Asura, Slice, Sl...|  [Asura, Slice, Sl...|
|Don't Breathe\n\n...|                  []|  [Don, Breathe\n\n

In [12]:
import nltk
#nltk.download('stopwords')
eng_stopwords = nltk.corpus.stopwords.words('english')

In [13]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

def remove_stop_word_from_movie_names(movies):
    if movies and len(movies[0].split()) == 1 and movies[0].lower() in eng_stopwords:
        return movies[1:]
    return movies

remove_stop_word_udf = udf(remove_stop_word_from_movie_names, ArrayType(StringType()))

In [14]:
df_final = df_final.withColumn("movie_names_final", remove_stop_word_udf(df_final["additional_movie_names"]))

### EXPLODING SUGGESTIONS 

In [16]:
from pyspark.sql.functions import explode, col, count, split

# Flatten the movie_names column
df_flattened = df_final.withColumn("movie_name", explode(col("movie_names_final")))

### GROUPING SUGGESTIONS

In [ ]:
df_movie_frequency = df_flattened.groupBy("movie_name").count()

In [17]:
from pyspark.sql.functions import desc

# Sort by count in descending order
df_top_1000_movies = df_movie_frequency.sort(desc('count')).limit(1000)

In [ ]:
df_top_1000_movies.show()

In [3]:
df_top_500_movies.write.parquet(f"s3a://{bucket}/projects/comments/extracted_movies")

# SPARK JOB

In [ ]:
# import sagemaker
# session = sagemaker.Session()
# bucket = "project-group34"
# !wget -qO- https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/jars/spark-nlp-assembly-5.1.3.jar | aws s3 cp - s3://{bucket}/lab8/spark-nlp-assembly-5.1.3.jar
# !aws s3 ls s3://{bucket}/lab8/spark-nlp-assembly-5.1.3.jar

In [2]:
!mkdir -p ./code

In [29]:
%%writefile ./code/suggestion_extract_process.py

import subprocess
import sys

import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print(sys.path)

subprocess.check_call([sys.executable, "-m", "pip", "install", "spark-nlp"])

import os
import logging
import argparse

# Import pyspark and build Spark session
from pyspark.sql.functions import *
from pyspark.sql.types import (
    DoubleType,
    IntegerType,
    StringType,
    StructField,
    StructType,
)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType
import re
from pyspark.sql.functions import explode, count
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

from pyspark.sql.functions import desc

import nltk
nltk.download('stopwords')
eng_stopwords = nltk.corpus.stopwords.words('english')

logging.basicConfig(format='%(asctime)s,%(levelname)s,%(module)s,%(filename)s,%(lineno)d,%(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

def main():
    
    parser = argparse.ArgumentParser(description="app inputs and outputs")
    parser.add_argument("--s3_dataset_path", type=str, help="Path of dataset in S3")    
    parser.add_argument("--col_name_for_filtering", type=str, help="Name of the column to filter")
    args = parser.parse_args()

    spark = SparkSession.builder \
    .appName("Spark NLP")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3")\
    .getOrCreate()
    
    logger.info(f"Spark version: {spark.version}")
    logger.info(f"sparknlp version: {sparknlp.version()}")
    
    # This is needed to save RDDs which is the only way to write nested Dataframes into CSV format
    sc = spark.sparkContext
    sc._jsc.hadoopConfiguration().set(
        "mapred.output.committer.class", "org.apache.hadoop.mapred.FileOutputCommitter"
    )

    # Downloading the data from S3 into a Dataframe
    logger.info(f"going to read {args.s3_dataset_path} for r/{args.col_name_for_filtering}")
    df = spark.read.parquet(args.s3_dataset_path, header=True)
    vals = [args.col_name_for_filtering]
    df_filtered = df.where(col("subreddit").isin(vals))
    logger.info(f"finished reading files...")
    
    # DATA CLEANING
    comments_filtered = df_filtered.filter((df.body != '[deleted]') & (df.author != '[deleted]'))
    comments_filtered_movies = comments_filtered.where(col("subreddit").isin("MovieSuggestions"))

    # Define the pipeline stages
    document_assembler = DocumentAssembler() \
        .setInputCol("body") \
        .setOutputCol("document")

    sentence_detector = SentenceDetector() \
        .setInputCols(["document"]) \
        .setOutputCol("sentence")

    tokenizer = Tokenizer() \
        .setInputCols(["sentence"]) \
        .setOutputCol("token")

    # Use a pretrained embeddings model, for example, BERT
    embeddings = BertEmbeddings.pretrained("bert_base_cased", "en") \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("embeddings")

    ner_model = NerDLModel.pretrained("ner_dl_bert", "en") \
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = NerConverter() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")

    # Build the pipeline
    nlp_pipeline = Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter
    ])

    # Apply the pipeline to your DataFrame
    model = nlp_pipeline.fit(comments_filtered_movies)
    result = model.transform(comments_filtered_movies)
    
    print("NLP Pipeline Ran Succesfully!")

    # Define a UDF to filter and extract movie names
    def extract_movies(chunks):
        movie_names = [chunk.result for chunk in chunks if chunk.metadata['entity'] in ['PERSON', 'ORG']]
        return movie_names

    extract_movie_names_udf = udf(extract_movies, ArrayType(StringType()))

    # Apply the UDF to the DataFrame
    movies_df = result.withColumn("movie_names", extract_movie_names_udf(F.col("ner_chunk")))


    @udf(StringType())
    def remove_movie_names_udf(text, movie_names):
        if movie_names:
            for name in movie_names:
                text = text.replace(name, ' ')
            return ' '.join(text.split())
        else:
            return text

    # UDF to extract movie names using regex
    @udf(ArrayType(StringType()))
    def extract_movie_names_regex_udf(text, movie_names):
        movie_name_pattern = r'(?:\"([^\"]+)\"|([A-Z][a-z]*(?:\s+(?:[a-z]+\s+)*[A-Z][a-z]*)*)(?: \(\d{4}\))?)'

        movie_matches = re.findall(movie_name_pattern, text)
        movies = [match[0] or match[1] or match[2] for match in movie_matches]
        return movie_names + movies
    
    def remove_stop_word_from_movie_names(movies):
        if movies and len(movies[0].split()) == 1 and movies[0].lower() in eng_stopwords:
            return movies[1:]
        return movies

    # Remove movie names from the 'body' text
    df_removed_movie_names = movies_df.withColumn("body_no_movies", remove_movie_names_udf(movies_df["body"], movies_df["movie_names"]))

    # The regex method to supplement the NER extraction
    df_final = df_removed_movie_names.withColumn("additional_movie_names", extract_movie_names_regex_udf(df_removed_movie_names["body_no_movies"], df_removed_movie_names["movie_names"]))

    df_final = df_final.select("subreddit", "author", "body", "parent_id", "id", "created_utc", "score", "controversiality", "additional_movie_names")
    
    print("Movie Names Extracted")
    
    remove_stop_word_udf = udf(remove_stop_word_from_movie_names, ArrayType(StringType()))

    df_final = df_final.withColumn("movie_names_final", remove_stop_word_udf(df_final["additional_movie_names"]))
    
    # Flatten the movie_names column
    df_flattened = df_final.withColumn("movie_name", explode(col("movie_names_final")))

    # Group by movie_name and count the occurrences
    df_frequency = df_flattened.groupBy("movie_name").agg(count("*").alias("frequency"))
    
    print("Aggregation Done!")
    
    # Sort the DataFrame by frequency in descending order and take the top 1000
    df_top_1000_movies = df_frequency.orderBy(desc("frequency")).limit(1000)
    
    # df_top_1000_movies_pd = df_top_1000_movies.toPandas()
    
    bucket = "project-group34"
    output_prefix_data_comments = "project/comments/"
    s3_path = f"s3a://{bucket}/{output_prefix_data_comments}" + args.col_name_for_filtering + "/"
    
    print(f"Writing to {s3_path}")
    df_top_1000_movies.write.parquet(s3_path)
    # Save the DataFrame in CSV format to S3
    #df_top_1000_movies.write.option("header", "true").mode("overwrite").csv(s3_path)
    print(f"Finished writing to {s3_path}")

    # df_top_1000_movies.to_csv(f"{s3_path}/movie_suggestions/{csv_name}")
    
    logger.info(f"all done...")
    
if __name__ == "__main__":
    main()

Overwriting ./code/suggestion_extract_process.py


In [10]:
%%time
import boto3
import sagemaker
from sagemaker.spark.processing import PySparkProcessor

account_id = boto3.client('sts').get_caller_identity()['Account']

CPU times: user 19.3 ms, sys: 317 µs, total: 19.6 ms
Wall time: 41.5 ms


In [11]:
account_id

'655678691473'

In [30]:
%%time
import time
import sagemaker
from sagemaker.spark.processing import PySparkProcessor

# Setup the PySpark processor to run the job. Note the instance type and instance count parameters. SageMaker will create these many instances of this type for the spark job.
role = sagemaker.get_execution_role()
spark_processor = PySparkProcessor(
    base_job_name="sm-spark-project",
    image_uri=f"{account_id}.dkr.ecr.us-east-1.amazonaws.com/sagemaker-spark:latest",
    framework_version="3.3",
    role=role,
    instance_count=8,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=7200,
)

# # S3 URI of the initialization script
# s3_uri_init_script = f's3://{bucket}/{script_key}'

# s3 paths
session = sagemaker.Session()
output_prefix_logs = f"spark_logs"

configuration = [
    {
        "Classification": "spark-defaults",
        "Properties": {"spark.executor.memory": "12g", "spark.executor.cores": "4"},
    }
]

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 110 ms, sys: 50 µs, total: 110 ms
Wall time: 203 ms


In [2]:
%%time
subreddit_list = ["MovieSuggestions", "televisionsuggestions", "Animesuggest"]
for subreddit in subreddit_list:
    print(f"going to extract suggestions data for subreddit={subreddit}")
    bucket = "project-group34"
    output_prefix_data_comments = "project/comments/yyyy=*"
    s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
    col_name_for_filtering = subreddit

    # run the job now, the arguments array is provided as command line to the Python script (Spark code in this case).
    spark_processor.run(
        submit_app="./code/suggestion_extract_process.py",
        submit_jars=[f"s3://{bucket}/spark-nlp-assembly-5.1.3.jar"],
        arguments=[
            "--s3_dataset_path",
            s3_path,
            "--col_name_for_filtering",
            col_name_for_filtering,
        ],
        spark_event_logs_s3_uri="s3://{}/{}/spark_event_logs".format(bucket, output_prefix_logs),
        logs=False,
        configuration=configuration
    )
    # give some time for resources from this iterations to get cleaned up
    # if we start the job immediately we could get insufficient resources error
    time.sleep(60)

# SENTIMENT ANALYSIS

In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,
                                LemmatizerModel, StopWordsCleaner)
import pyspark.sql.functions as F

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.executor.memory", "12g")\
    .config("spark.executor.cores", "3")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider"
    )\
    .getOrCreate()

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-971fa7b5-46e3-491f-82c8-ba2077ba4fee;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

3.4.0


In [5]:
reviews_df = pd.read_csv("../../data/csv/rotten_tomatoes_movie_reviews.csv")
movie_df = pd.read_csv("../../data/csv/rotten_tomatoes_movies.csv")

In [6]:
movie_df = movie_df[['id', 'title']]
reviews_df = reviews_df[['id', 'reviewText']]
#left outer join on id
merged_df = pd.merge(movie_df, reviews_df, on='id', how='left')
merged_df.isnull().sum()
merged_df = merged_df.dropna()

In [7]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    return text
# Clean the 'reviewText' column
merged_df['cleanedText'] = merged_df['reviewText'].apply(clean_text)

In [8]:
df = spark.createDataFrame(merged_df)

In [9]:
documentAssembler = DocumentAssembler()\
    .setInputCol("cleanedText")\
    .setOutputCol("document")
# Regex Tokenizer to break words
tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('token')
# Normalizing and setting case insensitive to be true
normalizer = Normalizer() \
     .setInputCols(['token']) \
     .setOutputCol('normalized') \
     .setLowercase(True)
# Lemmatizing
lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemma')
# finisher converts tokens to human-readable output
finisher = Finisher() \
     .setInputCols(['lemma']) \
     .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.
[ / ]

[OK!]


In [10]:
pipeline = Pipeline() \
     .setStages([
           documentAssembler,
           tokenizer,
           normalizer,
           lemmatizer,
           finisher
     ])

In [11]:
pipelineModel = pipeline.fit(df)
result = pipelineModel.transform(df)

In [12]:
result = result.withColumn("final_text", F.concat_ws(" ", "finished_lemma"))

In [13]:
documentAssembler = DocumentAssembler()\
    .setInputCol("final_text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

sentimentdl = SentimentDLModel.pretrained(name="sentimentdl_use_twitter", lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

sentimentdl1 = ClassifierDLModel.pretrained(name="classifierdl_use_emotion")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment_emotion")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl,
          sentimentdl1
      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
Download done! Loading the resource.
[ / ]

2023-11-21 04:43:41.782472: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ | ]sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
Download done! Loading the resource.
[OK!]
classifierdl_use_emotion download started this may take some time.
Approximate size to download 21.3 MB
[ | ]classifierdl_use_emotion download started this may take some time.
Approximate size to download 21.3 MB
Download done! Loading the resource.
[OK!]


In [14]:
sentiment_model = nlpPipeline.fit(result)
sentiment_result = sentiment_model.transform(result)

In [15]:
sentiment_result.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- cleanedText: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metada

In [16]:
finalresult = sentiment_result.select(
    F.explode(F.arrays_zip('sentiment.result', 'sentiment.metadata')).alias("cols"),
    F.expr("title").alias("title"),
    F.expr("cleanedText").alias("text")
)

In [17]:
finalresult.printSchema()

root
 |-- cols: struct (nullable = false)
 |    |-- result: string (nullable = true)
 |    |-- metadata: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)



In [18]:
from pyspark.sql.functions import col

# Selecting the necessary columns
finalresult = finalresult.select(
    col("cols.result").alias("sentiment"),
    col("cols.metadata")["positive"].alias("positive_score"),
    col("cols.metadata")["negative"].alias("negative_score"),
    "title",
    "text"
)

# # Showing the first few rows of the modified DataFrame
# finalresult.show(truncate=False)


In [19]:
# Group by the 'title' and calculate the average of 'positive_score' and 'negative_score'
avg_scores = finalresult.groupBy("title").agg(
    F.avg("positive_score").alias("average_positive_score"),
    F.avg("negative_score").alias("average_negative_score")
)

In [20]:
# Get top 20 movies with highest average positive scores
top_20_positive = avg_scores.orderBy(F.desc("average_positive_score")).limit(20)

# Get top 20 movies with highest average negative scores
top_20_negative = avg_scores.orderBy(F.desc("average_negative_score")).limit(20)

In [24]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [25]:
top_20_positive_pd = top_20_positive.toPandas()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
top_20_negative_pd = top_20_negative.toPandas()